In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import pandas as pd
import numpy as np
from scipy.spatial import distance as dist
from math import hypot
import math
import winsound
frequency=2500
duration=500

In [2]:
new_model=tf.keras.models.load_model("my_mask_model.h5")

In [3]:
facecascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [4]:
def facemask(frame):
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=facecascade.detectMultiScale(gray,1.1,4)
    font=cv2.FONT_HERSHEY_PLAIN
    m=0
    for x,y,w,h in faces:
        roi_gray=gray[y:y+h, x:x+w]
        roi_color=frame[y:y+h, x:x+w]
        facess=facecascade.detectMultiScale(roi_gray)
        if len(facess)!=0:
            for (ex,ey,ew,eh) in facess:
                face_roi=roi_color[ey:ey+eh,ex:ex+ew]
                final_image=cv2.resize(face_roi,(224,224))
                final_image=np.expand_dims(final_image,axis=0)
                Predictions=new_model.predict(final_image)
                if (Predictions>0.5):
                    status="No mask"
                    cv2.putText(frame,status,(x+6,y-6),font,1,(0,0,255),1)
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
                    winsound.Beep(frequency,duration)
                    m=m+1
                else:
                    status="Face mask"
                    cv2.putText(frame,status,(x+6,y-6),font,1,(0,255,0),1)
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)  
        
    text1 = "Mask Violations: {}".format(m)
    cv2.putText(frame, text1, (10, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 2)
    
    return frame

In [5]:
labelsPath = "./coco.names"
LABELS = open(labelsPath).read().strip().split("\n")
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")
weightsPath = "./yolov3.weights"
configPath = "./yolov3.cfg"
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [6]:
def person(image):  
    (H, W) = image.shape[:2]
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)
    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > 0.8 and classID == 0:
                box = detection[0:4]* np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.1,0.064)
    ind = []
    for i in range(0,len(classIDs)):
        if(classIDs[i]==0):
            ind.append(i)
            
    a = []
    b = []
    color = (0,255,0)
    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            a.append(int((x+x+w)/2))
            b.append(int((y+y+h)/2))
            #cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            
    distance=[] 
    nsd = []
    for i in range(0,len(a)-1):
        for k in range(1,len(a)):
            if(k==i):
                break
            else:
                x_dist = (a[k] - a[i])
                y_dist = (b[k] - b[i])
                d = math.sqrt(x_dist * x_dist + y_dist * y_dist)
                distance.append(d)
                if(d<=300):
                    nsd.append(i)
                    nsd.append(k)
                else:
                    (x,y)=(boxes[i][0],boxes[i][1])
                    (w,h)=(boxes[i][2],boxes[i][3])
                    (x1,y1)=(boxes[k][0],boxes[k][1])
                    (w1,h1)=(boxes[k][2],boxes[k][3])
                    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                    cv2.rectangle(image,(x1,y1),(x1+w1,y1+h1),color,2)
                nsd = list(dict.fromkeys(nsd))
    color = (0, 0, 255)
    
    for i in nsd:
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    text="Physical distance violation={}".format(len(nsd))
    cv2.putText(image, text, (10, 100), cv2.FONT_HERSHEY_SIMPLEX,0.85, color, 2)
    
    return image

In [7]:
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
import tkinter
from PIL import Image
from PIL import ImageTk

In [9]:
#facemask and social distance
class Root(Tk):
    def __init__(self):
        super(Root,self).__init__()
        self.title("Social Distance and Facemask Detection")
        self.minsize(640,400)
        self.labelFrame=ttk.LabelFrame(self,text="Detect in an image")
        self.labelFrame.grid(column=0,row=1,padx=20,pady=20)
        self.button()
        self.labelFrame1=ttk.LabelFrame(self,text="Detect in a video")
        self.labelFrame1.grid(column=1,row=1,padx=20,pady=20)
        self.button1()
        self.labelFrame2=ttk.LabelFrame(self,text="Detect in Webcam")
        self.labelFrame2.grid(column=2,row=1,padx=20,pady=20)
        self.button2()
        
    def button(self):
        self.button=ttk.Button(self.labelFrame,text="Browse an image",command=self.fileDialog)
        self.button.grid(column=1,row=1)
        self.button=ttk.Button(self.labelFrame,text="Process",command=self.imageprocess)
        self.button.grid(column=2,row=1)
    
    def button1(self):
        self.button1=ttk.Button(self.labelFrame1,text="Browse a video",command=self.fileDialog1)
        self.button1.grid(column=4,row=1)
        self.button1=ttk.Button(self.labelFrame1,text="Process",command=self.videoprocess)
        self.button1.grid(column=5,row=1)
        
    def button2(self):
        self.button2=ttk.Button(self.labelFrame2,text="Webcam",command=self.webcam)
        self.button2.grid(column=7,row=1)
        
        
        
    def fileDialog(self):
        self.filename=filedialog.askopenfilename(initialdir='/',title="Select a file",filetype=(("jpeg files",'*.jpg'),("png","*.png")))
        self.label=ttk.Label(self.labelFrame,text="")
        self.label.grid(column=1,row=2)
        self.label.configure(text=self.filename)
        
    def fileDialog1(self):
        self.filename1=filedialog.askopenfilename(initialdir='/',title="Select a file",filetype=(("jpeg files",'*.mp4'),("all","*.*")))
        self.label1=ttk.Label(self.labelFrame1,text="")
        self.label1.grid(column=4,row=2)
        self.label1.configure(text=self.filename1)
        
    def imageprocess(self):
        self.img=cv2.imread(self.filename)
        self.output=facemask(self.img)
        self.output=person(self.output)
        cv2.imshow('detection',self.output)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    def videoprocess(self):
        self.cap=cv2.VideoCapture(self.filename1)
        while True:
            _,self.frame=self.cap.read()
            self.output=facemask(self.frame)
            self.output=person(self.output)
            cv2.imshow('detection',self.output)
            if cv2.waitKey(1)==13:
                break
        self.cap.release()
        cv2.destroyAllWindows()
        
    def webcam(self):
        self.cap=cv2.VideoCapture(0)
        while True:
            _,self.frame=self.cap.read()
            self.output=facemask(self.frame)
            self.output=person(self.output)
            cv2.imshow('webcam detection',self.output)
            if cv2.waitKey(1)==13:
                break
        self.cap.release()
        cv2.destroyAllWindows()
        
        
        
if __name__=='__main__':
    root=Root()
    root.mainloop()
        

In [ ]:
def facemask_dist(frame):
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=facecascade.detectMultiScale(gray,1.1,4)
    centroids=[]
    MIN_DISTANCE=200
    font=cv2.FONT_HERSHEY_PLAIN
    violate=set()
    m=0
    for x,y,w,h in faces:
        cv2.rectangle(frame,(x+1,y+1),(x+1+w,y+1+h),(255,0,0),1)
        center_x=int((x+(x+w))/2)
        center_y=int((y+(y+h))/2)
        centroids.append((center_x,center_y))
        roi_gray=gray[y:y+h, x:x+w]
        roi_color=frame[y:y+h, x:x+w]
        facess=facecascade.detectMultiScale(roi_gray)
        if len(facess)!=0:
            for (ex,ey,ew,eh) in facess:
                face_roi=roi_color[ey:ey+eh,ex:ex+ew]
                final_image=cv2.resize(face_roi,(224,224))
                final_image=np.expand_dims(final_image,axis=0)
                Predictions=new_model.predict(final_image)
                if (Predictions>0.5):
                    status="No mask"
                    cv2.putText(frame,status,(x+6,y-6),font,1,(0,0,255),1)
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
                    m=m+1
                else:
                    status="Face mask"
                    cv2.putText(frame,status,(x+6,y-6),font,1,(0,255,0),1)
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)  
        if len(centroids)>=2:    
            for i in range(len(centroids)-1):
                for j in range(i+1,len(centroids)):
                    D=hypot(centroids[i][0]-centroids[j][0],centroids[i][1]-centroids[j][1])
                    if D<MIN_DISTANCE:
                        violate.add(i)
                        violate.add(j)
                        cv2.circle(frame,centroids[i],5,(0,0,255),-1)
                        cv2.putText(frame,'FDV',(centroids[i][0]+11,centroids[i][1]+11),font,1,(0,0,255),2)
                        cv2.circle(frame,centroids[j],5,(0,0,255),-1)
                        cv2.putText(frame,'FDV',(centroids[j][0]+11,centroids[j][1]+11),font,1,(0,0,255),2)
                        cv2.line(frame,centroids[i],centroids[j],(0,0,255),1)
         
    text = "Face Distancing Violations(FDV): {}".format(len(violate))
    cv2.putText(frame, text, (10, 25),cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 2)
    text1 = "Mask Violations: {}".format(m)
    cv2.putText(frame, text1, (10, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 2)
    
    return frame

In [12]:
#face distance
class Root(Tk):
    def __init__(self):
        super(Root,self).__init__()
        self.title("Social Distance and Facemask Detection")
        self.minsize(640,400)
        self.labelFrame=ttk.LabelFrame(self,text="Detect in an image")
        self.labelFrame.grid(column=0,row=1,padx=20,pady=20)
        self.button()
        self.labelFrame1=ttk.LabelFrame(self,text="Detect in a video")
        self.labelFrame1.grid(column=1,row=1,padx=20,pady=20)
        self.button1()
        self.labelFrame2=ttk.LabelFrame(self,text="Detect in Webcam")
        self.labelFrame2.grid(column=2,row=1,padx=20,pady=20)
        self.button2()
        
    def button(self):
        self.button=ttk.Button(self.labelFrame,text="Browse an image",command=self.fileDialog)
        self.button.grid(column=1,row=1)
        self.button=ttk.Button(self.labelFrame,text="Process",command=self.imageprocess)
        self.button.grid(column=2,row=1)
    
    def button1(self):
        self.button1=ttk.Button(self.labelFrame1,text="Browse a video",command=self.fileDialog1)
        self.button1.grid(column=4,row=1)
        self.button1=ttk.Button(self.labelFrame1,text="Process",command=self.videoprocess)
        self.button1.grid(column=5,row=1)
        
    def button2(self):
        self.button2=ttk.Button(self.labelFrame2,text="Webcam",command=self.webcam)
        self.button2.grid(column=7,row=1)
        
        
        
    def fileDialog(self):
        self.filename=filedialog.askopenfilename(initialdir='/',title="Select a file",filetype=(("jpeg files",'*.jpg'),("png","*.png")))
        self.label=ttk.Label(self.labelFrame,text="")
        self.label.grid(column=1,row=2)
        self.label.configure(text=self.filename)
        
    def fileDialog1(self):
        self.filename1=filedialog.askopenfilename(initialdir='/',title="Select a file",filetype=(("jpeg files",'*.mp4'),("all","*.*")))
        self.label1=ttk.Label(self.labelFrame1,text="")
        self.label1.grid(column=4,row=2)
        self.label1.configure(text=self.filename1)
        
    def imageprocess(self):
        self.img=cv2.imread(self.filename)
        self.output=facemask_dist(self.img)
        cv2.imshow('detection',self.output)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    def videoprocess(self):
        self.cap=cv2.VideoCapture(self.filename1)
        while True:
            _,self.frame=self.cap.read()
            self.output=facemask_dist(self.frame)
            cv2.imshow('detection',self.output)
            if cv2.waitKey(1)==13:
                break
        self.cap.release()
        cv2.destroyAllWindows()
        
    def webcam(self):
        self.cap=cv2.VideoCapture(0)
        while True:
            _,self.frame=self.cap.read()
            self.output=facemask_dist(self.frame)
            cv2.imshow('webcam detection',self.output)
            if cv2.waitKey(1)==13:
                break
        self.cap.release()
        cv2.destroyAllWindows()
        
        
        
if __name__=='__main__':
    root=Root()
    root.mainloop()
        